In [1]:
import psycopg2
import datetime
import re

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT


In [2]:
vector_dec_line_pattern = re.compile("^vector")
vector_res_line_pattern = re.compile("^\d+")

In [3]:
config_name = "test"
now = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
now = "01_01_21_00_00_00"

In [23]:
# Setup a connection to postgres and ultimately setup a new DB for the run

con = psycopg2.connect("user=postgres password='simserver'")

con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Obtain a DB Cursor

cursor    = con.cursor()

db_name   = "{}_{}".format(config_name, now)

# Create table statement

sqlCreateDatabase = "create database {};".format(db_name)

cursor.execute(sqlCreateDatabase)

cursor.close()

con.commit()

con.close()

In [4]:
all_stats = [
    "messageArrival",
        "sciSent",
        "tbSent",
        "txRxDistanceSCI",
        "txRxDistanceTB",
        "sciReceived",
        "sciDecoded",
        "sciFailedHalfDuplex",
        "tbReceived",
        "tbDecoded",
        "tbFailedDueToNoSCI",
        "tbFailedHalfDuplex",
        "tbFailedButSCIReceived",
        "periodic",
        "sciUnsensed",
        "missedTransmission",
        "generatedGrants",
        "selectedSubchannelIndex",
        "selectedNumSubchannels",
        "grantBreak",
        "grantBreakMissedTrans",
        "grantBreakTiming",
        "grantBreakSize",
        "resourceReselectionCounter",
        "retainGrant",
        "tbFailedDueToProp",
        "tbFailedDueToInterference",
        "sciFailedDueToProp",
        "sciFailedDueToInterference",
        "cbr",
        "cbrPscch",
        "subchannelReceived",
        "subchannelsUsed",
        "senderID",
        "subchannelsUsedToSend",
        "subchannelSent",
        "grantStartTime",
        "takingReservedGrant",
        "awareness1sStat",
        "awareness500msStat",
        "awareness200msStat",
        "selectedSubchannelIndex",
        "selectedNumSubchannels",
        "macNodeID",
        "posX",
        "posY",
        "interPacketDelay",
        "oneShot",
        "oneShotSent",
        "oneShotReceived",
        "oneShotDecoded",
        "oneShotFailedHalfDuplex",
        "oneShotFailedDueToProp",
        "oneShotFailedDueToInterference",
        "oneShotUnsensed",
        "txRxDistanceOneShot",
        "rriChange"
]
grant_info = [
    "missedTransmission",
    "generatedGrants",
    "selectedSubchannelIndex",
    "selectedNumSubchannels",
    "grantBreak",
    "grantBreakMissedTrans",
    "grantBreakTiming",
    "grantBreakSize",
    "resourceReselectionCounter",
    "retainGrant",
    "grantStartTime",
    "takingReservedGrant",
    "selectedSubchannelIndex",
    "selectedNumSubchannels",
    "rriChange"
]

reception_info = [
    "txRxDistanceSCI",
    "txRxDistanceTB",
    "sciReceived",
    "sciDecoded",
    "sciFailedHalfDuplex",
    "tbReceived",
    "tbDecoded",
    "tbFailedDueToNoSCI",
    "tbFailedHalfDuplex",
    "tbFailedButSCIReceived",
    "periodic",
    "sciUnsensed",
    "tbFailedDueToProp",
    "tbFailedDueToInterference",
    "sciFailedDueToProp",
    "sciFailedDueToInterference",
    "subchannelReceived",
    "subchannelsUsed",
    "senderID",
    "oneShot",
    "oneShotSent",
    "oneShotReceived",
    "oneShotDecoded",
    "oneShotFailedHalfDuplex",
    "oneShotFailedDueToProp",
    "oneShotFailedDueToInterference",
    "oneShotUnsensed",
    "txRxDistanceOneShot"
]

send_info = [
    "sciSent",
    "tbSent",
    "subchannelsUsedToSend",
    "subchannelSent"
]

misc = [
    "messageArrival",
    "macNodeID",
    "interPacketDelay",
    "awareness1sStat",
    "awareness500msStat",
    "awareness200msStat"
]

cbr = [
    "cbr",
    "cbrPscch"
]

position = [
    "posX",
    "posY"
]

In [34]:
commands = [    
"""
CREATE TABLE grants (
    Time FLOAT(3), 
    NodeID INT,
    run INT,
    missedTransmission INT,
    generatedGrants INT,
    selectedSubchannelIndex INT,
    selectedNumSubchannels INT,
    grantBreak INT,
    grantBreakMissedTrans INT,
    grantBreakTiming INT,
    grantBreakSize INT,
    resourceReselectionCounter INT,
    retainGrant INT,
    grantStartTime INT,
    takingReservedGrant INT,
    rriChange INT,
    PRIMARY KEY (Time,NodeID, run)
)""",
"""
CREATE TABLE receptions (
    Time FLOAT(3), 
    NodeID INT,
    run INT,
    txRxDistanceSCI INT,
    txRxDistanceTB INT,
    sciReceived INT,
    sciDecoded INT,
    sciFailedHalfDuplex INT,
    tbReceived INT,
    tbDecoded INT,
    tbFailedDueToNoSCI INT,
    tbFailedHalfDuplex INT,
    tbFailedButSCIReceived INT,
    periodic INT,
    sciUnsensed INT,
    tbFailedDueToProp INT,
    tbFailedDueToInterference INT,
    sciFailedDueToProp INT,
    sciFailedDueToInterference INT,
    subchannelReceived INT,
    subchannelsUsed INT,
    senderID INT,
    PRIMARY KEY (Time,NodeID,run)
)""",
"""
CREATE TABLE transmissions (
    Time FLOAT(3), 
    NodeID INT,
    run INT,
   
    sciSent INT,
    tbSent INT,
    subchannelsUsedToSend INT,
    subchannelSent INT,
   
    PRIMARY KEY (Time,NodeID, run)
)""",
"""
CREATE TABLE positions (
    Time FLOAT(3), 
    NodeID INT,
    run INT,
    posX INT,
    posY INT,
    
    PRIMARY KEY (Time,NodeID,run)
)""",
"""
CREATE TABLE cbr (
    Time FLOAT(3), 
    NodeID INT,
    run INT,
    
    cbr FLOAT(4),
    cbrPscch FLOAT(4),
  
    PRIMARY KEY (Time,NodeID, run)
)""",
"""
CREATE TABLE misc (
    Time FLOAT(3), 
    NodeID INT,
    run INT,
  
    messageArrival INT,
    macNodeID INT,
    interPacketDelay FLOAT(3),
    awareness1sStat FLOAT(4),
    awareness500msStat FLOAT(4),
    awareness200msStat FLOAT(4),
    PRIMARY KEY (Time,NodeID,run)
)""",
"""
CREATE TABLE config (
    id serial NOT NULL PRIMARY KEY,
    parameters json NOT NULL
)"""
]

In [35]:
try:
    conn = psycopg2.connect(
        host="localhost",
        database=db_name,
        user="postgres",
        password="simserver")

    cur = conn.cursor()
    # create table one by one
    for command in commands:
        cur.execute(command)
    # close communication with the PostgreSQL database server
    cur.close()
    # commit the changes
    conn.commit()
    
except (Exception, psycopg2.DatabaseError) as error:
        print(error)
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')

Database connection closed.


In [110]:
def remove_vectors(json_fields, single=False):
    # Simple function to remove the vector from results in json file might be we remove that part from the json file
    if single:
        return json_fields.replace(":vector", "")
    for i in range(len(json_fields)):
        json_fields[i] = json_fields[i].replace(":vector", "")
    return json_fields

def parse_vector_desc_line(line):
    # Converts a vector description line to a dictionary for use in parsing later
    node_id_pattern = re.compile("\[\d+\]")

    vector_line_dict = {"nodeID": None, "vectorName": None, "ETV": True}
    split_line = line.split(" ")
    vector_num = int(split_line[1])
    match = node_id_pattern.search(split_line[2])
    nodeID = int(match.group().strip("[]"))
    vector_name = split_line[3]
    vector_name = vector_name.split(":")[0]
    if "ETV" in split_line[4]:
        ETV = True
    else:
        ETV = False

    vector_line_dict["nodeID"] = nodeID
    vector_line_dict["vectorName"] = vector_name
    vector_line_dict["ETV"] = ETV

    return vector_num, vector_line_dict

def parse_vector_line(line):
    # Simple function to split a vector line and convert to floats.
    try:
        line = bytes(line, 'utf-8').decode('utf-8', 'ignore')
        split_nums = line.split()
        for i in range(len(split_nums)):
            split_nums[i] = float(split_nums[i])
        return split_nums
    except ValueError as e:
        print("Line: {} could not be converted due to bad encoding".format(line))
        return

def prepare_results(vector_dict, vector_id, parsed_vec, results):
    # Parses the vector line information to be written to the csv file.
    node_id = vector_dict[vector_id]["nodeID"]
    vector_name = vector_dict[vector_id]["vectorName"]
    if vector_dict[vector_id]["ETV"]:
        time = parsed_vec[2]
        value = parsed_vec[3]
    else:
        time = parsed_vec[1]
        value = parsed_vec[2]
        
    stat_group = "misc"
    if vector_name in grant_info:
        stat_group = "grants"
    elif vector_name in reception_info:
        stat_group = "receptions"
    elif vector_name in send_info:
        stat_group = "transmissions"
    elif vector_name in cbr:
        stat_group = "cbr"
    elif vector_name in position:
        stat_group = "position"    
        
    if time in results:
        if node_id in results[time]:
            if stat_group in results[time][node_id]:
                if vector_name not in results[time][node_id][stat_group]:
                    results[time][node_id][stat_group][vector_name] = []
            else:
                results[time][node_id][stat_group] = {
                    vector_name : []
                }
        else:
            results[time][node_id] = {
                stat_group: {
                    vector_name : []
                }
            }
    else:
        results[time] = {
            node_id: {
                stat_group: {
                    vector_name : []
                }
            }
        }
    results[time][node_id][stat_group][vector_name].append(value)

    return


In [111]:
def process_chunk(lines, vector_dict, no_interest_vectors, stats, results):
    
    # Whats needed 
    # Dictionary Time -> NodeID -> Grant, reception, transmission, position, cbr, misc
    # grant -> "stat_x": [], "stat_y": [], "stat_z": []
    
    print("begin processing chunk")

    for line in lines:
        if vector_dec_line_pattern.match(line):
            # if line matches a vector declaration, parse the vector description
            vector_num, vec_dict = parse_vector_desc_line(line)
            if vector_num is None and vec_dict is None:
                continue
            if vec_dict["vectorName"] in stats:
                # Vector is of interest, add it to our overall dictionary and update it's index.
                vector_dict[vector_num] = vec_dict
            else:
                # Mark this as a vector we don't care about.
                no_interest_vectors[vector_num] = None

        elif vector_res_line_pattern.match(line):
            # {"nodeID": None, "vectorName": None, "ETV": True} This is what it looks like
            parsed_vec = parse_vector_line(line)
            # If the previous step fails then we can simply continue to the next line ignoring this line.
            if parsed_vec is None:
                continue
            vector_id = parsed_vec[0]
            if vector_id in vector_dict:
                # Write out to a csv file correctly
                prepare_results(vector_dict, vector_id, parsed_vec, results)

            else:
                if vector_id not in no_interest_vectors:
                    # Write the line out in case we found it before declaration. Only if it is of possible interest.
                    print("There was a line which appeared in the wrong place: {}".format(line))

    return results


In [112]:
def read_vector_file(vector_path, stats):
    # Reads the csv file, parses it and writes to a temp file for use later in generating a DF and CSV file.
    vector_dict = {}
    no_interest_vectors = {}  # Probably don't need to remember one's we don't care for.

    unfinished_results = {}

    vector_file = open(vector_path, "r")

    # Read 100,000 lines
    tmp_lines = vector_file.readlines(1000000)
    while tmp_lines and chunk_results == {}:
        chunk_results = process_chunk([line for line in tmp_lines], vector_dict,
                                      no_interest_vectors, stats, unfinished_results)
        
        # Prepare to write the data into the database
        
        # Need to ensure that the groups have the correct length otherwise we need to keep them for the next chunk
        # If they are incorrect they go into unfinished results
        
        # Read the next 100,000 lines
        tmp_lines = vector_file.readlines(1000000)

    vector_file.close()
    
    return chunk_results

In [113]:
vector_path = "/Users/brianmccarthy/git_repos/results-analysis/data/omnet/congestion-control/rri-crlimit-20pc-2021-05-11-21_33_02/run-1.vec"

In [114]:
chunk_results = read_vector_file(vector_path, all_stats)

begin processing chunk
begin processing chunk


In [115]:
chunk_results

{500.0: {0: {'position': {'posX': [384.04136846054], 'posY': [10.0]}},
  1: {'position': {'posX': [371.77565572598], 'posY': [10.0]}},
  2: {'position': {'posX': [258.06158795681], 'posY': [10.0]}},
  3: {'position': {'posX': [245.56927821808], 'posY': [10.0]}},
  4: {'position': {'posX': [232.76458086051], 'posY': [10.0]}},
  5: {'position': {'posX': [220.11405090605], 'posY': [10.0]}},
  6: {'position': {'posX': [207.41377632643], 'posY': [10.0]}},
  7: {'position': {'posX': [194.18561909839], 'posY': [10.0]}},
  8: {'position': {'posX': [179.81105679563], 'posY': [10.0]}},
  9: {'position': {'posX': [164.51190140156], 'posY': [10.0]}},
  10: {'position': {'posX': [152.13510176372], 'posY': [10.0]}},
  11: {'position': {'posX': [139.56375644163], 'posY': [10.0]}},
  12: {'position': {'posX': [358.9101714203], 'posY': [10.0]}},
  13: {'position': {'posX': [125.91656522561], 'posY': [10.0]}},
  14: {'position': {'posX': [111.7839760544], 'posY': [10.0]}},
  15: {'position': {'posX': [9